In [ ]:
# DAQ code WIP
import PyDAQmx as nidaq, numpy as np, time
from scipy import signal
runlength = 60
fs = 1000
nchans = 2
t = np.linspace(0, runlength, runlength*fs, endpoint=False)
stim = signal.square(2 * np.pi * t)
t1 = nidaq.Task()
t1.CreateAIVoltageChan("Dev2/ai0", None, nidaq.DAQmx_Val_Diff, -10, 10, nidaq.DAQmx_Val_Volts, None) # Whisker stim IN
t1.CreateAIVoltageChan("Dev2/ai1", None, nidaq.DAQmx_Val_Diff, -10, 10, nidaq.DAQmx_Val_Volts, None) # LED Strobe
t2 = nidaq.Task()
t2.CreateAOVoltageChan("Dev2/ao0", None, -10, 10, nidaq.DAQmx_Val_Volts, None) # Whisker stim OUT

t1.CfgSampClkTiming("", fs, nidaq.DAQmx_Val_Rising, nidaq.DAQmx_Val_FiniteSamps, fs*runlength*nchans)
t1.StartTask()
t2.StartTask()
print(time.time())
t2.WriteAnalogF64(len(stim),
                  0,
                  10.0,
                  0,
                  stim,
                  None,
                  None)
print(time.time())


In [ ]:
import numpy as np
data = np.zeros((fs*runlength*nchans), dtype=np.float64)
read = nidaq.int32()
t1.ReadAnalogF64(fs*runlength, 5, nidaq.DAQmx_Val_GroupByChannel,
   data, len(data)*nchans, nidaq.byref(read), None)

In [ ]:
# Plot AI channels
%matplotlib inline
import matplotlib.pyplot as plt
i = 1
plt.plot(data[fs*runlength*i:fs*runlength*(i+1)],label = 'AI'+str(i))
    
plt.legend()
plt.show()